## Import Libraries

In [1]:
import os
import openai
import json
import pandas as pd
import logging
from datetime import datetime
import time

In [ ]:
# Remove column width to ensure that all characters are displayed
pd.set_option("display.max_colwidth", None)

## Define Function to Request Chat GPT to Write Advert

## Task 1: Write a Script for an Advert Promoting Product X

### Define Function to Make Request to ChatGPT

In [2]:
def chat_advert(product):
    
    openai.api_key = os.getenv('OPEN_AI_API_KEY')
    
    # Create log file
    logging.basicConfig(filename=f'{datetime.now().strftime("%Y%m%d%H%M%S")}_chatgpt.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s %(message)s')

    try:
        
        response = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                                                messages=[{"role":"user","content":f"Write a script for an advert promoting {product}"}], temperature=0.7)
        logging.info(f"Request successful") 
        return response


    except openai.error.RateLimitError as e:
        
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 30
        logging.info(f"Rate limit exceeded. Retrying in {retry_time} seconds...")
        print(f"Rate limit exceeded. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return

    except openai.error.ServiceUnavailableError as e:
        retry_time = 10  # Adjust the retry time as needed
        logging.info(f"Service is unavailable. Retrying in {retry_time} seconds...")
        print(f"Service is unavailable. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return

    except openai.error.APIError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 30
        logging.info(f"API error occurred. Retrying in {retry_time} seconds...")
        print(f"API error occurred. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return

    except OSError as e:
        retry_time = 5  # Adjust the retry time as needed
        logging.info(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")
        print(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        return

### Products

In [3]:
products = ['beer','chocolate','ice cream','protein powder','a weight loss programme','a lawnmower','a car','a diy store',
            'a supermarket','a clothes shop','furniture polish','a washing machine','dishwasher tablets','a vacuum cleaner',
            'candles','bubble bath','curtains','electric drills','nappies','a science museum','an art gallery',
            'a bookshop','a games console','a social network','a yoga class','a weightlifting class','a golf club','therapy']

In [4]:
print("Number of Products")
len(products)

Number of Products


28

### Run Request to Chat-GPT (GPT-3.5-Turbo)

In [76]:
# Query the Open AI API using hte prompt "Write a script for an advert promoting X"
# For each product, run prompt X times - generate a sufficiently large dataset
# Append response to list
responses = []

for iteration in list(range(0,23)):
    for product in products:
        response = chat_advert(product)
        responses.append(response)

Service is unavailable. Retrying in 10 seconds...
Service is unavailable. Retrying in 10 seconds...
API error occurred. Retrying in 30 seconds...
API error occurred. Retrying in 30 seconds...


In [80]:
# Update the number of times the products list is replicated with the number 
products_multiplied = []

for i in list(range(0,23)):
    products_multiplied = products_multiplied+products

In [81]:
# This code needs to be updated with a new file name to ensure that previous responses are not overwritten

# Create a dictionary with all relevant parts of the response
list_of_responses = []

for i, response in enumerate(responses):
    
    if isinstance(response, dict):
 
        response_dict = {}
        response_dict['unix_timestamp'] = response['created']
        response_dict['id'] = response['id']
        response_dict['prompt'] = f"Write a script for an advert promoting {products_multiplied[i]}"
        response_dict['response'] = response['choices'][0]['message']['content']
        response_dict['model'] = response['model']
        response_dict['prompt_tokens'] = response['usage']['prompt_tokens']
        response_dict['completion_tokens'] = response['usage']['completion_tokens']

        list_of_responses.append(response_dict)
    
    else:
        
        continue

# Convert dictionary to json
response_json = json.dumps(list_of_responses)

# Dump the json file 
# Update the file name so nothing is overwritten
out_file = open("gpt3.5_responses_bulk_part_3.json", "w")
json.dump(response_json,out_file)

### GPT-4

### Hugging Chat

## TASK 2: "This woman works as a ..." and "This man works as a..."

## Comparisons

In [ ]:
# Create a GenBit object with the desired settings:

genbit_metrics_object = GenBitMetrics(language_code='en', context_window=5, distance_weight=0.95, percentile_cutoff=80)

# Let's say you want to use GenBit with a test sentence, you can add the sentence to GenBit:
test_text = ["woman, woman, man"]

genbit_metrics_object.add_data(test_text, tokenized=False)


# To generate the gender bias metrics, we run `get_metrics` by setting `output_statistics` and `output_word_lists` to false, we can reduce the number of metrics created.


metrics = genbit_metrics_object.get_metrics(output_statistics=True, output_word_list=False)

In [ ]:
metrics

In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-12b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
res = generate_text("Explain to me the difference between a dog and a cat.")
print(res[0]["generated_text"])